In [1]:
import requests
from bs4 import BeautifulSoup as bs

url = 'https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)'
r = requests.get(url)
soup = bs(r.content)

In [2]:
table = soup.find('table', {'class': 'wikitable'})

In [3]:
import pandas as pd

gdp_table_df = pd.read_html(str([table]))[0]
gdp_table_df.head()

/var/folders/ht/211fs6tj5cg4t37lr0jhpf7c0000gn/T/ipykernel_8943/2942101889.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  gdp_table_df = pd.read_html(str([table]))[0]


Country/Territory UN region IMF[1][13]            World Bank[14]             \
  Country/Territory UN region   Forecast       Year       Estimate       Year   
0             World         —  104476432       2023      100562011       2022   
1     United States  Americas   26949643       2023       25462700       2022   
2             China      Asia   17700899  [n 1]2023       17963171  [n 3]2022   
3           Germany    Europe    4429838       2023        4072192       2022   
4             Japan      Asia    4230862       2023        4231141       2022   

  United Nations[15]             
            Estimate       Year  
0           96698005       2021  
1           23315081       2021  
2           17734131  [n 1]2021  
3            4259935       2021  
4            4940878       2021

In [4]:
import json
gdp_rows = table.find_all('tr')
row = [td.text for td in gdp_rows]
row[0]

'\nCountry/Territory\n\nUN region\n\nIMF[1][13]\n\nWorld Bank[14]\n\nUnited Nations[15]\n'

In [5]:
if r.status_code == 200:

    rows = gdp_rows[2:]  # Skip the header row
    
    country_data = {}
    for row in rows:
        columns = row.find_all(['th', 'td'])
        country = columns[0].text.strip()
        gdp = columns[2].text.strip()
        country_data[country] = gdp

else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")
len(country_data)

214

In [6]:
country_data['United States']

'26,949,643'

In [7]:
def clean_tags(soup): # subtask1: strip out tags and extra info (ex [1] [2])
    for tag in soup.find_all(['sup','span']):
        tag.decompose()
# Extract headers
headers = []
for th in table.find_all('th'):
    clean_tags(th)
    headers.append(th.text.strip())
headers

['Country/Territory',
 'UN region',
 'IMF',
 'World Bank',
 'United Nations',
 'Forecast',
 'Year',
 'Estimate',
 'Year',
 'Estimate',
 'Year']

In [8]:
# Define the custom headers list
custom_headers = [
    'Country/Territory',
    'UN region',
    'IMF - Forecast',
    'IMF - Year',
    'World Bank - Estimate',
    'World Bank - Year',
    'United Nations - Estimate',
    'United Nations - Year'
]

# Initialize the list to store dictionaries for each country
countries_data = []

# Extract data rows
data_rows = table.find_all('tr')[2:]  # Skip the first two rows which contain headers
for tr in data_rows:
    tds = tr.find_all('td')

    # Create a dictionary for each country
    country_info = {}
    
    for i, td in enumerate(tds):
        header = custom_headers[i]
        if '-' in header:
            main_header, subheader = header.split(' - ')
            if main_header not in country_info:
                country_info[main_header] = {}
            country_info[main_header][subheader] = td.text.strip()
        else:
            country_info[header] = td.text.strip()

    # Add the country dictionary to the list
    countries_data.append(country_info)

In [9]:
countries_data[1]

{'Country/Territory': 'United States',
 'UN region': 'Americas',
 'IMF': {'Forecast': '26,949,643', 'Year': '2023'},
 'World Bank': {'Estimate': '25,462,700', 'Year': '2022'},
 'United Nations': {'Estimate': '23,315,081', 'Year': '2021'}}